In [5]:
from DiSuQ.Torch import models as TorchModels
from DiSuQ.Kerman import models as KermanModels

from torch import tensor
from numpy import arange,linspace,var,logspace
from DiSuQ.utils import plotCompare
from time import perf_counter
import memory_profiler as mp
import pandas

import torch,numpy
from threadpoolctl import threadpool_limits
from DiSuQ.utils import plotCompare

In [6]:
def torchInstantiationFluxonium(N,El=10.,Ec=100.,Ej=50.,sparse=False):
    basis = {'O':[N],'I':[],'J':[]}; rep = 'K'
    circuit = TorchModels.fluxonium(basis,El=El,Ec=Ec,Ej=Ej,sparse=sparse)
    flux_point = {'I':tensor(0.)}
    H_LC = circuit.kermanHamiltonianLC()
    H_J = circuit.kermanHamiltonianJosephson(flux_point)
    return H_LC+H_J

def torchInstantiationTransmon(N,El=10.,Ec=100.,Ej=50.,sparse=False):
    basis = {'O':[N],'I':[],'J':[]}; rep = 'Q'; basis = [N]
    circuit = TorchModels.transmon(basis,Ec=Ec,Ej=Ej,sparse=sparse)
    flux_point = {'I':tensor(0.)}
    H_LC = circuit.chargeHamiltonianLC()
    H_J = circuit.josephsonCharge(flux_point)
    return H_LC+H_J

def numpyInstantiation(N,El=10.,Ec=100.,Ej=50,sparse=False):
    basis = {'O':[N],'I':[],'J':[]}; rep = 'K'
    circuit = KermanModels.fluxonium(basis,El=El,Ec=Ec,Ej=Ej)
    flux_point = {'I':0.}
    H_LC = circuit.kermanHamiltonianLC()
    H_J = circuit.kermanHamiltonianJosephson(flux_point)
    return H_LC+H_J

In [7]:
threads = [16,32,48,64]
n_bases = 7

In [8]:
dSpa = []
vectors = False
bases = logspace(2,3.2,n_bases,dtype=int)
print(bases)
arguments = {'El':10.,'Ec':100.,'Ej':50.}
for n_threads in threads:
    print(n_threads,'----------')
    for N in bases:
        print(N,'==============')
        torch.set_num_threads(n_threads)
        
        H = torchInstantiationTransmon(N,sparse=False).to(float)
        start = perf_counter()
        mem = mp.memory_usage((torch.lobpcg,(H,4)))
        time = perf_counter()-start
        dSpa.append({'lib':'torch','stage':'diag','hilbert':N,'threads':n_threads,
                        'peak_memory':max(mem)-min(mem),'time':time,'vectors':vectors,'rep':'dense'})
                              
        H = torchInstantiationTransmon(N,sparse=True).to(float)
        start = perf_counter()
        mem = mp.memory_usage((torch.lobpcg,(H,4)))
        time = perf_counter()-start
        dSpa.append({'lib':'torch','stage':'diag','hilbert':N,'threads':n_threads,
                        'peak_memory':max(mem)-min(mem),'time':time,'vectors':vectors,'rep':'sparse'})
                              
        
dSpa = pandas.DataFrame(dSpa)
dSpa.to_csv('sparse-dense-re.csv')

[ 100  158  251  398  630 1000 1584]
16 ----------
100 ==============
158 ==============
251 ==============
398 ==============
630 ==============
1000 ==============
1584 ==============
32 ----------
100 ==============
158 ==============
251 ==============
398 ==============
630 ==============
1000 ==============
1584 ==============
48 ----------
100 ==============
158 ==============
251 ==============
398 ==============
630 ==============
1000 ==============
1584 ==============
64 ----------
100 ==============
158 ==============
251 ==============
398 ==============
630 ==============
1000 ==============
1584 ==============


-------------------------

In [27]:
dLog = []
vectors = False
bases = logspace(1,4,n_bases,dtype=int)
print(bases)
arguments = {'El':10.,'Ec':100.,'Ej':50.}
for n_threads in threads:
    print(n_threads,'----------')
    for N in bases:
        print(N,'==============')
        torch.set_num_threads(n_threads)
        H = torchInstantiation(N)
        start = perf_counter()
        mem = mp.memory_usage((torch.linalg.eigvalsh,(H,)))
        time = perf_counter()-start
        dLog.append({'lib':'torch','stage':'diag','hilbert':N,'threads':n_threads,
                        'peak_memory':max(mem)-min(mem),'time':time,'vectors':vectors,'rep':'dense'})

        #mkl.set_num_threads(n_threads)
        with threadpool_limits(limits=n_threads, user_api='blas'):
            H = numpyInstantiation(N)        
            start = perf_counter()
            mem = mp.memory_usage((numpy.linalg.eigvalsh,(H,)))
            time = perf_counter()-start
            dLog.append({'lib':'numpy','stage':'diag','hilbert':N,'threads':n_threads,
                         'peak_memory':max(mem)-min(mem),'time':time,'vectors':vectors,'rep':'dense'})

dLog = pandas.DataFrame(dLog)
dLog.to_csv('parallelization.csv')

[   10    56   316  1778 10000]
16 ----------
10 ==============
56 ==============
316 ==============
1778 ==============
10000 ==============
32 ----------
10 ==============
56 ==============
316 ==============
1778 ==============
10000 ==============
64 ----------
10 ==============
56 ==============
316 ==============
1778 ==============
10000 ==============


In [42]:
dLog.to_csv('parallelization.csv')

# Sparse

# Parallelization

In [28]:
#dLog = pandas.read_csv('parallelization.csv')

In [29]:
dNum = dLog[dLog['lib']=='numpy']
dTor = dLog[dLog['lib']=='torch']

In [30]:
dNum.groupby(['threads','hilbert'])['time'].apply(list)

threads  hilbert
16       10         [0.32048748806118965]
         56          [0.2620991840958595]
         316        [0.24262834712862968]
         1778        [0.5671932324767113]
         10000          [42.567871209234]
32       10          [0.2676594518125057]
         56          [0.2741946466267109]
         316        [0.29015256091952324]
         1778        [0.5784363113343716]
         10000        [34.15516912564635]
64       10          [0.2774595655500889]
         56         [0.27154920995235443]
         316        [0.28771113976836205]
         1778        [0.7344554513692856]
         10000       [43.694167755544186]
Name: time, dtype: object

In [31]:
dTor.groupby(['threads','hilbert'])['time'].apply(list)

threads  hilbert
16       10          [0.3507659137248993]
         56           [0.360497422516346]
         316        [0.24776870384812355]
         1778       [0.42042311280965805]
         10000        [77.66510659456253]
32       10         [0.40429428964853287]
         56         [0.23862718418240547]
         316        [0.23803506419062614]
         1778        [0.4247867427766323]
         10000       [53.645921275019646]
64       10         [0.43307001143693924]
         56          [0.2745783366262913]
         316        [0.28994231671094894]
         1778        [0.5305673442780972]
         10000        [85.90453988313675]
Name: time, dtype: object

In [ ]:
plotCompare(bases,{'numpy':dNum[dLog['']]})

# Multi-Node GPU

# I/O bottleneck

# GPU advantage